## Challenge TelecomX-2-ML – Análisis Predictivo de Churn

### Objetivo

En esta segunda etapa del Challenge de Data Science de Alura, el enfoque se centra en el **modelado predictivo** para anticipar la evasión de clientes (Churn) en TelecomX.
Luego de completar el proceso de **ETL y Análisis Exploratorio (EDA)**, el objetivo ahora es construir, evaluar y optimizar modelos de Machine Learning que permitan:

* Predecir qué clientes tienen mayor probabilidad de abandonar el servicio.
* Identificar las variables más influyentes en la decisión de churn.
* Proporcionar recomendaciones estratégicas basadas en evidencia predictiva.


Flujo de implementación:

1. Feature Engineering
2. Preprocesamiento
3. Train/Test SplitPipeline
4. Cross-Validation (en Train)
5. Comparación de modelos
6. Optimización
7. Entrenamiento final
8. Evaluación en Test

---



## Implementación del Modelado Predictivo

### 1 Ingeniería de Características (Feature Engineering)

En esta fase se optimizan y crean variables para mejorar la capacidad predictiva del modelo.

* **Creación de `num_services`**: Variable que cuantifica la intensidad de uso del cliente. Un mayor número de servicios puede reflejar mayor fidelización o mayor complejidad de abandono.
* **Variables derivadas**: Posible creación de ratios como gasto promedio por servicio.
* **Eliminación de variables irrelevantes o redundantes**:

  * `CustomerID` (identificador sin valor predictivo).
  * Análisis de variables altamente correlacionadas que puedan generar multicolinealidad.
* **Análisis de importancia preliminar** para validar aporte de nuevas variables.

---

### 2 Preprocesamiento de Datos (Data Preprocessing)

Preparación técnica de los datos para su uso en modelos de Machine Learning.

* **Codificación de Variables Categóricas**:

  * One-Hot Encoding para variables nominales (ej. `PaymentMethod`).
  * Label Encoding para la variable objetivo (`Churn`: 0 = No, 1 = Sí).

* **Escalado de Variables Numéricas**:

  * Estandarización (StandardScaler) o Normalización (MinMaxScaler).
  * Aplicado a variables como:

    * `Tenure`
    * `ChargesMonthly`
    * `num_services`

* **Pipeline de Preprocesamiento**:
  Implementación de `Pipeline` para evitar data leakage y garantizar reproducibilidad.

---

### 3 División del Dataset

Separación estratégica para validar el desempeño real del modelo.

* **Definición de variables**:

  * `X` → Variables predictoras.
  * `y` → Variable objetivo (Churn).

* **Train/Test Split**:

  * 80% entrenamiento
  * 20% prueba
  * Uso de `stratify=y` para mantener proporción de churn.

---

### 4 Validación Cruzada (Cross-Validation)

Antes de evaluar el modelo final, se implementa validación cruzada:

* **K-Fold Cross Validation (k=5 o 10)**
  Permite:

  * Reducir varianza en la estimación del rendimiento.
  * Detectar sobreajuste.
  * Obtener métricas promedio más robustas.

Esta etapa es clave para asegurar estabilidad del modelo, se aplica sólo alconjunto de entrenamiento.

---

### 5 Selección y Entrenamiento de Modelos

Se exploran diferentes algoritmos para comparar desempeño:

* **Regresión Logística**

  * Modelo base interpretable.
  * Buena referencia inicial.

* **Random Forest**

  * Maneja relaciones no lineales.
  * Robusto ante outliers y ruido.

* **Gradient Boosting (ej. XGBoost o similares)**

  * Alto poder predictivo.
  * Captura patrones complejos.

* **LightGBM**

  * Más rápido y eficiente que otros boosting

  * Excelente rendimiento en datasets estructurados

  * Maneja bien variables categóricas codificadas


Cada modelo se entrena utilizando validación cruzada para comparación objetiva.

---

### 6 Evaluación de Métricas

Dado que el problema es de clasificación binaria con posible desbalance, se priorizan las siguientes métricas:

* **Matriz de Confusión**

  * Falsos Positivos (FP)
  * Falsos Negativos (FN)

* **Recall (Sensibilidad)**

  * Métrica crítica en churn: identificar correctamente a quienes se irán.

* **Precision**

  * Minimizar falsas alarmas.

* **F1-Score**

  * Balance entre Precision y Recall.

* **ROC-AUC**

  * Evaluación global del poder discriminatorio del modelo.

- En `variables` como churn, **Recall** tiene mayor relevancia en resultados.

---

### 7 Optimización de Hiperparámetros

Para maximizar el rendimiento:

* **GridSearchCV**
* **RandomizedSearchCV**

Aplicado sobre el modelo con mejor desempeño preliminar.

Objetivo:

* Ajustar profundidad de árboles.
* Número de estimadores.
* Regularización.
* Learning rate (en boosting).

Todo validado mediante Cross-Validation.

---

### 8 Modelo Final y Análisis de Importancia

* Entrenamiento final con mejores hiperparámetros.
* Evaluación sobre conjunto de prueba.
* Análisis de importancia de variables.
* Interpretación estratégica de resultados.

---

## Resultados Esperados

* Modelo robusto y validado.
* Identificación de variables clave en churn.
* Segmentación de clientes en riesgo.
* Base técnica para implementar estrategias de retención.

---

## Impacto Estratégico

Este modelo predictivo permite:

* Implementar campañas de retención dirigidas.
* Optimizar recursos comerciales.
* Reducir tasa de churn.
* Incrementar el Customer Lifetime Value (CLV).


In [ ]:
# ==============================
# CONFIGURACIÓN GLOBAL
# ==============================

# Ignorar warnings irrelevantes
import warnings
#warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# ==============================
# BIBLIOTECAS PARA MANEJO DE DATOS
# ==============================

import requests
import pandas as pd
import numpy as np
import json
import os

# ==============================
# BIBLIOTECAS PARA PREPROCESAMIENTO - ML
# ==============================

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# ==============================
# BIBLIOTECAS PARA VISUALIZACIÓN
# ==============================

import matplotlib.pyplot as plt
import seaborn as sns

# ==============================
# CONFIGURACIÓN DE VISUALIZACIONES
# ==============================

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Si trabajas en Jupyter, puedes activar esto:
# %matplotlib inline

# ==============================
# CONFIGURACIÓN DE PANDAS
# ==============================

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# ==============================
# PALETA DE COLORES PERSONALIZADA
# ==============================

color_no_churn = '#2C3E50'  # Azul oscuro elegante
color_churn = '#E74C3C'     # Rojo elegante

color_palette = [
  '#F8F9FA', # Fondo Limpio (Ghost White)
  '#E9ECEF', # Bordes/Secciones (Slate Gray)
  '#22577A', # Títulos/Estructura (Dark Imperial Blue)
  '#38A3A5', # Procesos activos (Cadet Blue)
  '#57CC99', # Éxito/Validación (Emerald)
  '#80ED99'  # Acentos ligeros (Light Green)
]

sns.set_palette(color_palette)

# ==============================
# CREAR CARPETA PARA IMÁGENES
# ==============================

os.makedirs('imgs', exist_ok=True)

print('✅ Bibliotecas importadas y entorno configurado correctamente')

## Carga de datos

Previo análisis con dataset generado en la 1ra etapa y al no encontrar mayores inconsistencias, se utiliza el dataset proprocionado por Alura One "dataset_tratado.csv". 

In [ ]:
url = "https://raw.githubusercontent.com/LenninTemoche/Challenge-TelecomX-2-Alura-One-DS/refs/heads/main/datos_tratados.csv"

df = pd.read_csv(url, sep=",")

df.info()


In [ ]:
# 1. Definimos las columnas que representan servicios
service_cols = [
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# 2. Función para contar los servicios activos de cada cliente
def count_services(row):
    count = 0
    # InternetService: Si es 'DSL' o 'Fiber optic' cuenta como 1 (si es 'No', es 0)
    if row['InternetService'] != 'No':
        count += 1
    
    # Para el resto, sumamos 1 si el valor es 'Yes'
    for col in service_cols:
        if col != 'InternetService' and row[col] == 'Yes':
            count += 1
    return count

# Aplicamos la función para crear la nueva columna
df['num_services'] = df.apply(count_services, axis=1)

print(f"✅ Nueva columna creada: 'num_services' (Rango: {df['num_services'].min()} - {df['num_services'].max()} servicios)")

# 1. Definimos las columnas que representan servicios
service_cols = [
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# 2. Función para contar los servicios activos de cada cliente
def count_services(row):
    count = 0
    # InternetService: Si es 'DSL' o 'Fiber optic' cuenta como 1 (si es 'No', es 0)
    if row['InternetService'] != 'No':
        count += 1
    
    # Para el resto, sumamos 1 si el valor es 'Yes'
    for col in service_cols:
        if col != 'InternetService' and row[col] == 'Yes':
            count += 1
    return count

# Aplicamos la función para crear la nueva columna
df['num_services'] = df.apply(count_services, axis=1)

print(f"✅ Nueva columna creada: 'num_services' (Rango: {df['num_services'].min()} - {df['num_services'].max()} servicios)")